正則表達式:TVBS清理作者格式方法

In [31]:
import re
#作者欄位清理方法，要放author[0]
def tvbs_author_etl(author_text):
    author = []
    author_text = re.sub(r"記者  ", "", author_text)
    author_text = re.sub(r" / 攝影.*", "", author_text)
    author_text = re.sub(r" / ", "", author_text)
    author_text = re.sub(r"編輯  ", "", author_text)
    author_text = re.sub(r" 報導", "", author_text)
    author_text = re.sub(r" ", ",", author_text)
    #如果有多位作者，使用、斷開，並一一取出放進list
    if author_text.find(",") != -1:
        author_text = author_text.split(",")
        for a in author_text:
            author.append(a)
    elif author_text == "":
        author = []
    else:
        author.append(author_text)
    return author

正則表達式:SETN清理作者格式方法

In [65]:
#SETN作者欄位清理方法，要放author[0]
def setn_author_etl(author_text):
    author = []
    #將／XX報導XX字串清除
    author_text = re.sub(r"／.*報導.*", "", author_text)
    #將／XX特稿XX字串清除
    author_text = re.sub(r"／.*特稿.*", "", author_text)
    #將記者字串清除
    author_text = re.sub(r"記者", "", author_text)
    #將文／字串清除
    author_text = re.sub(r"文／", "", author_text)
    #如果有多位作者，使用、斷開，並一一取出放進list
    if author_text.find("、") != -1:
        author_text = author_text.split("、")
        for a in author_text:
            author.append(a)
    elif author_text == "":
        author = []
    else:
        author.append(author_text)
    return author

正則表達式:ETtoday清理作者格式方法

In [49]:
#ETtoday作者欄位清理方法，要放author[0]
def ettoday_author_etl(author_text):
    author = []
    author_text = re.sub("／.?.?報導.*", "", author_text)
    author_text = re.sub(".*記者", "", author_text)
    author_text = re.sub("\xa0", "", author_text)
    #如果有多位作者，使用、斷開，並一一取出放進list
    if author_text.find("、") != -1:
        author_text = author_text.split("、")
        for a in author_text:
            author.append(a)
    elif author_text == "":
        author = []
    else:
        author.append(author_text)
    return author

文章正反面情緒分析 需要的套件

In [33]:
!pip install udicOpenData
!pip install Swinger
#在dockerfile的指令
#ADD ./dict_all.txt /opt/conda/lib/python3.6/site-packages/udicOpenData/dictionary/
from udicOpenData.dictionary import *
jieba.load_userdict(os.path.join(DIR_NAME, 'dict_all.txt'))
#要等一下子
from Swinger import Swinger

文章正反面情緒分析方法

In [55]:
s = None
def emo_swinger(content, model='LogisticRegression'):
    global s
    if s == None:
        s = Swinger()
        s.load(model)
    # default model= "LogisticRegression", 可替換為"MultinomialNB"
    emo_result = s.swing(content)
    if emo_result == "pos":
        result = "positive"
    elif emo_result == "neg":
        result = "negative"
    return result

文章關鍵字 需要的套件

In [64]:
import jieba.analyse

文章關鍵字方法

In [62]:
def keyword_analysis(content):
    #default kewwords = 8
    kws =jieba.analyse.extract_tags(content, 5)
    return kws

文章藍綠貼標分析 需要的套件

In [35]:
!pip install pandas 
import jieba
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.externals import joblib

將jieba斷詞寫成方法

In [36]:
def cutflow(p):
    cutresult = " ".join(jieba.cut(p))
    return cutresult.replace("\r", "").replace("\n", "")

載入詞向量模型與貝氏模型

In [37]:
clg = joblib.load('clg') 
vec = joblib.load('vec') 

文章藍綠貼標分析方法:貝氏定理

In [38]:
def label_predict(text):
    docs_news = cutflow(text)
    c=[{"content" : docs_news,
       "tag" : 0}]
    d = pd.DataFrame(c)
    test_counts = vec.transform(d["content"])
    pre = clg.predict(test_counts)
    label_result = list(pre)[0]
    if label_result == '1':
        result = "blue"
    elif label_result == '0':
        result = "green"
    return result

文章摘要分析 需要的套件

In [39]:
!pip install networkx
!pip install textrank4zh
from  __future__  import print_function
import jieba
import numpy
import networkx
import sys
import codecs 
from textrank4zh import TextRank4Keyword ,TextRank4Sentence

文章摘要方法:TextRank

In [40]:
def news_summary(text):
    tr4s = TextRank4Sentence()
    tr4s.analyze( text = text, lower = True , source = ' all_filters ' )
    for item in tr4s.get_key_sentences( num = 1 ):
        return item.sentence

Elasticsearch 需要的套件

In [41]:
!pip install elasticsearch
from elasticsearch import Elasticsearch

In [42]:
# 建立Elasticsearch連線
es = Elasticsearch(['elasticsearch:9200'])

In [ ]:
# 在Elasticsearch建立index，亦可加入body參數先定義mapping格式
es.indices.create(index = 'news', ignore = 400)

In [ ]:
# 將Elasticsearch定義所放入的資料格式:mapping
es.indices.put_mapping(index = "news",
                       doc_type = "politics",                
                       body = {
                        "properties": {
                            "source": {"type": "text"},
                            "url": {"type": "text"},
                            "title": {"type": "text"},
                            "date_": {"type": "text"},
                            "author": {"type": "text"},
                            "content": {"type": "text"},
                            "kw": {"type": "text"},
                            "img_url": {"type": "text"},
                            "@timestamp": {"type": "date"}
                        }
                    }
                )

In [43]:
# 為了將匯入資料的時間加入時區
from datetime import datetime, timedelta, timezone
tz_utc_8 = timezone(timedelta(hours=8)) # 創建時區UTC+8:00

Kafka Consumer需要的套件

In [44]:
!pip install kafka
from kafka import KafkaConsumer, TopicPartition
import sys
import json, time, requests

In [60]:
if __name__ == "__main__":
    # 步驟1.設定要連線到Kafka集群的相關設定, 產生一個Kafka的Consumer的實例
    consumer = KafkaConsumer(
        # Kafka集群在那裡?
        bootstrap_servers=["kafka1:29092"],
        # ConsumerGroup的名稱
        group_id="test01",
        # 指定msgKey的反序列化器, 若Key為None, 無法反序列化
        key_deserializer=bytes.decode,
        # 指定msgValue的反序列化器
        #value_deserializer=bytes.decode,
        value_deserializer=lambda m: json.loads(m.decode('utf-8')),
        # 是否從這個ConsumerGroup尚未讀取的partition / offset開始讀
        auto_offset_reset="earliest",
    )
    # 步驟2.指定想要訂閱訊息的topic名稱
    topic_name = "test"
    # 步驟3.讓Consumer向Kafka集群訂閱指定的topic
    consumer.subscribe(topics = topic_name)


    # 步驟4.持續的拉取Kafka有進來的訊息
    try:
        print("Start listen incoming messages ...","\n")
        # 持續監控是否有新的record進來 - 方法一
        for record in consumer:
            topic = record.topic
            partition = record.partition
            offset = record.offset
            timestamp = record.timestamp
        
            # 取出msgKey與msgValue
            msgKey = record.key
            msgValue = record.value
            
            # 將新聞作者欄位做處理後再放回
            if msgValue["source"] == "TVBS":      
                msgValue["author"] = tvbs_author_etl(msgValue["author"][0])
            elif msgValue["source"] == "SETN":
                msgValue["author"] = setn_author_etl(msgValue["author"][0])
            elif msgValue["source"] == "ETtoday":
                msgValue["author"] = ettoday_author_etl(msgValue["author"][0])
            
            # 將新聞內文做文章情緒分析並加入欄位與結果
            msgValue["ariticle_emotion"] = emo_swinger(msgValue["content"])
            # 將新聞內文做關鍵字分析再合併至新聞抓到的關鍵字列表，並把重複的篩選掉
            content_keyword = keyword_analysis(msgValue["content"])
            for k in content_keyword:
                msgValue["kw"].append(k)
            msgValue["kw"] = list(set(msgValue["kw"]))
            # 將新聞內文做藍綠分析並加入欄位結果
            msgValue["label"] = label_predict(msgValue["content"])
            # 將新聞做文章摘要並加入欄位與結果
            msgValue["abstract"] = news_summary(msgValue["content"])
            
            # 秀出metadata與msgKey & msgValue訊息
            print("topic=%s, partition=%s, offset=%s : (key=%s, value=%s)" % (record.topic, record.partition,
                                                                              record.offset, record.key, msgValue))
            print("\n")
            
            print("Start sending data to MySQL !!!")
            # 將資料送入MySQL
            ip_location = 'chatbot_api'
                         
            News = msgValue
            
            # 將json傳回API Server
            Endpoint = 'http://%s:5000/news' % (ip_location)

            # header要特別註明是json格式
            Header = {'Content-Type':'application/json'}
            
            # 傳送post對API server新增資料 
            Response = requests.post(Endpoint,headers=Header,data=json.dumps(News))

            # 印出Response的資料訊息
            print(Response)
            Response = Response.json()
            print(Response)
            print("\n")
            
        
            print("Start sending data to elasticsearch !!!")
            # 將資料送入Elasticsearch
            elasticsearch_data = msgValue
            elasticsearch_data["@timestamp"] = datetime.now().replace(tzinfo = tz_utc_8)            
            # 將資料放入Elasticsearch，若沒有index，會直接建立index、type 
            es.index(index = 'news', doc_type = 'politics', body = elasticsearch_data, refresh = True )
            print("\n")

    except:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
    finally:
        consumer.close()

Start listen incoming messages ... 

load default bestMainFeatures
load default bestMainFeatures success!!


/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DictVectorizer from version 0.18.1 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.18.1 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.18.1 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


load model from LogisticRegression
topic=test, partition=0, offset=9 : (key=ettoday, value={'source': 'ETtoday', 'url': 'https://www.ettoday.net/news/20181224/1339020.htm', 'title': '同意和卓榮泰直播 館長：我歸覽趴火笑你不敢來', 'date_': '2018/12/24 12:05', 'author': ['政治中心'], 'content': '民進黨黨主席候選人、行政院秘書長卓榮泰日前曾表示，若成吉思汗健身俱樂部館長陳之漢願邀他一起直播，他很願意聊聊，盼能聽到「同溫層聽不到的聲音」。對此，館長回應，卓榮泰沒跟他聯繫，「我笑你不敢來」，「我龜懶趴火，我很想要問一些問題」，要卓榮泰別只是透過媒體放話。據《鏡週刊》報導，卓榮泰認為，想要聆聽支持者的聲音，「我可以去跟陳其邁喝咖啡，我也可以去跟館長聊一聊啊！我可以唱詹雅雯的〈深情海岸〉，我也可以找鄭進一來唱〈家後〉吧！」對此，館長在直播中表示，卓榮泰沒有與他聯絡，自己很想跟一個要做黨主席的人一起直播，「我第一個就問促轉會」、「我很想問你，你當黨主席的時候，你的立委能不能通過一些有意義的審查案」、「我笑你不敢來！你要來不用新聞叫囂、直接打電話來嘛！」館長指出，若一起直播，大家就想看新任黨主席能不能讓他問一些問題，「我龜懶趴火，我很想要問一些問題」，像是薪水都沒漲，原物料一直漲，年輕人連摩托車都快買不起，同樣的機車在國外賣三分之一的價錢，「請問我們親愛的民進黨做了嗎？你們是上來圖利財團，還是為人民做事的？」', 'kw': ['對此', '我龜', '館長', '直播', '卓榮泰', '問題', '聲音', '民進黨'], 'img_url': 'https://cdn2.ettoday.net/images/3785/d3785468.jpg', 'ariticle_emotion': 'negative', 'label': 'green', 'abstract': '」對此，館長在直播中表示，卓榮泰沒有與他聯絡，自己很想跟一個要做黨主席的人一起直播，「我第一個就問促轉會」、「我很想問你，你當黨主席的時候，你的立委能不能通過一些有意義

<Response [200]>
{'status_describe': 'success add News'}


Start sending data to elasticsearch !!!


topic=test, partition=0, offset=16 : (key=ettoday, value={'source': 'ETtoday', 'url': 'https://www.ettoday.net/news/20181223/1338704.htm', 'title': '大學姊悄轉戰台中！柯P「她沒跟我講」…盧秀燕「曝光隱瞞原因」', 'date_': '2018/12/23 23:01', 'author': ['政治中心'], 'content': '準台中市長盧秀燕23日公布第5波人事，觀光局長由台北市長柯文哲競選辦公室發言人、「大學姊」林筱淇出任，成了這波人事的最大亮點；柯文哲得知後坦言，「她（林筱淇）去台中事先沒有跟我講！」對此，盧秀燕也緩頰，表示人事在發布前有要求林筱淇保密，因此她才未事先告知柯P。「用人不分黨派！」盧秀燕表示，先前就曾注意到林筱淇的口條清晰、態度穩健，對她留下很好的印象，因此主動與她聯繫，原本屬意要讓林擔任台中市府發言人，但在知道她的專長是行銷管理後，認為是個拚經濟、拚觀光的好人才，因此改讓她出任觀光局長。面對記者提問，是否有事先告知柯市長；林筱淇表示，先前曾於臉書發文找工作，這一事有告知柯P，「他知道我在找工作，但不知道後續找到怎樣的工作，相信今天記者會後就會知道我的新工作了！」現年40歲的林筱淇已婚，先生是台中市的開業醫師。林筱淇說，台中市是她的第二故鄉，原本新工作就鎖定在台北、台中兩地，當時盧秀燕主動與她聯繫感到相當訝異，「我跟盧市長沒有見過面、也不認識，但2人聊過後感受到她的溫暖、親切，稍加評估後決定接受挑戰！」對於這一人事案，柯文哲23日晚間受訪時表示，先前有詢問林筱淇要不要去選立委，但她意願似乎不高，「後來她說要去找工作，就這樣，她去台中事先沒有跟我講。盧秀燕也幫忙緩頰，表示人事發布前都有先要求當事人保密，避免受到不必要的干擾，維持一個純淨的空間、確保發布人事順利成功，「這也是林筱淇沒有跟柯報告新工作的原因。」', 'kw': ['台中市', '觀光局', '發布', '人事', '盧秀燕', '沒有